In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
from sklearn import preprocessing
import numpy as np
from statsmodels.stats import inter_rater as irr
pd.options.display.max_colwidth = 240
from sklearn.preprocessing import OneHotEncoder

In [2]:
juanjo = pd.read_csv('../Data/Data_Annotated/latest_juanjo_annotations.csv', sep = '\t')
juanma = pd.read_csv('../Data/Data_Annotated/latest_juanma_annotations.csv', sep = '\t')
ger = pd.read_csv('../Data/Data_Annotated/latest_ger_annotations.csv', sep = '\t')

In [3]:
juanjo = juanjo.rename(columns = {'sentimento':'sentiment_juanjo', 'relevancia': 'relevancia_juanjo', 'emocion': 'emocion_juanjo', 'ironia': 'ironia_juanjo'})
juanma = juanma.rename(columns = {'sentimento':'sentiment_juanma', 'relevancia': 'relevancia_juanma', 'emocion': 'emocion_juanma', 'ironia': 'ironia_juanma'})
ger = ger.rename(columns = {'sentimento':'sentiment_ger', 'relevancia': 'relevancia_ger', 'emocion': 'emocion_ger', 'ironia': 'ironia_ger'})

In [4]:
merged_df = pd.merge(pd.merge(juanjo[['text', 'tweet_id', 'sentiment_juanjo']],
                     juanma[['tweet_id', 'sentiment_juanma']], on=['tweet_id']),
                    ger[['tweet_id', 'sentiment_ger']], on=['tweet_id'])

In [5]:
dict_emociones = {
    'alegria' : ['Alegria', 'Confianza', 'Admiración','Agrado'],
    'miedo' : ['Miedo', 'Incertidumbre'],
    'sorpresa' : ['Sorpresa', 'Asombro'],
    'tristeza' : ['Tristeza' , 'Decepción'],
    'asco' : ['Asco', 'Desagrado'],
    'ira' : ['Ira', 'Odio'],
    'otra' : ['Otra'] }

In [6]:
for i  in dict_emociones.keys():
    merged_df[i + '_juanma'] = merged_df['sentiment_juanma'].str.contains('|'.join(dict_emociones[i]))
    merged_df[i + '_juanjo'] = merged_df['sentiment_juanjo'].str.contains('|'.join(dict_emociones[i]))
    merged_df[i + '_ger'] = merged_df['sentiment_ger'].str.contains('|'.join(dict_emociones[i]))

In [71]:
merged_df = merged_df.fillna(False)
merged_df['labels_author'] = merged_df.apply(lambda x: ', '.join(x.index[x == True]), axis=1)

In [88]:
for emotion in dict_emociones.keys():
    merged_df[emotion] =  merged_df[[emotion + '_juanjo',  emotion + '_ger',  emotion + '_juanma']].sum(axis=1)
    merged_df[emotion] = np.where(merged_df[emotion]>= 2, True, False)
merged_df['labels'] = merged_df[list(dict_emociones.keys())].apply(lambda x: ', '.join(x.index[x == True]), axis=1)
merged_df['labels_encoded'] = np.where(merged_df[list(dict_emociones.keys())] == 1, 1, 0).tolist()
labeled_df = merged_df[['text', 'tweet_id', 'labels', 'labels_encoded']]
labeled_df = labeled_df[labeled_df['labels'] != ''].reset_index(drop = True)
labeled_df.to_csv('../Data/Data_Annotated/test_labeled_dataset.csv', index = False)

In [6]:
labeled_df = pd.read_csv('../Data/Data_Annotated/test_labeled_dataset.csv')
labeled_df_hugging =  labeled_df[['text', 'labels_encoded']].rename(columns = {'labels_encoded' : 'label'})
labeled_df_hugging.to_csv('../Data/Data_Annotated/test_labeled_dataset_hugging.csv', index = False)